In [3]:
import requests
import os
import yaml
import pandas as pd

- Import credentials

In [4]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/lending_club_api.yml')))

In [5]:
account_id = credentials['account_id']

- Make the API call

In [77]:

r = requests.get('https://api.lendingclub.com/api/investor/v1/loans/listing',
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [78]:
r

<Response [200]>

In [79]:
json_response =  r.json()

In [80]:
json_response.keys()

[u'loans', u'asOfDate']

In [81]:
test_loan = json_response['loans'][50]

In [83]:
test_loan['homeOwnership']

u'RENT'

- Convert to DF

In [52]:
loans_df = pd.DataFrame(json_response['loans'])

#### Convert EmpLength to years

In [53]:
loans_df.empLength = loans_df.empLength.apply(lambda x: x/60 if x >0 else 0)

#### Create month to int 1-12

In [54]:
loans_df['month'] = loans_df.acceptD.apply(lambda x: pd.to_datetime(x).month)

#### Crea cols term_36 and term_60

In [58]:
loans_df['term_36'] = [1 if i ==36 else 0 for i in loans_df.term]

In [59]:
loans_df['term_60'] = [1 if i ==60 else 0 for i in loans_df.term]

#### Create grade columns
- a:g

In [63]:
loans_df['grade_a'] = [1 if i =='A' else 0 for i in loans_df.grade]
loans_df['grade_b']= [1 if i =='B' else 0 for i in loans_df.grade]
loans_df['grade_c']= [1 if i =='C' else 0 for i in loans_df.grade]
loans_df['grade_d']= [1 if i =='D' else 0 for i in loans_df.grade]
loans_df['grade_e']= [1 if i =='E' else 0 for i in loans_df.grade]
loans_df['grade_f']= [1 if i =='F' else 0 for i in loans_df.grade]
loans_df['grade_g']= [1 if i =='G' else 0 for i in loans_df.grade]

### Create home ownership columns
-'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',

In [ ]:
loans_df['home_ownership_any'] = [1 if i =='ANY' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_mortgage'] = [1 if i =='MORTGAGE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_none'] = [1 if i =='NONE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_other'] = [1 if i =='OTHER' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_own'] = [1 if i =='OWN' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_rent'] = [1 if i =='RENT' else 0 for i in loans_df.homeOwnership]

In [56]:
loans_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 104 columns):
accNowDelinq                  int64
accOpenPast24Mths             int64
acceptD                       object
addrState                     object
addrZip                       object
allUtil                       float64
annualInc                     float64
annualIncJoint                float64
applicationType               object
avgCurBal                     int64
bcOpenToBuy                   float64
bcUtil                        float64
chargeoffWithin12Mths         int64
collections12MthsExMed        int64
creditPullD                   object
delinq2Yrs                    int64
delinqAmnt                    float64
desc                          object
dti                           float64
dtiJoint                      float64
earliestCrLine                object
empLength                     float64
empTitle                      object
expD                          object
exp

In [ ]:
# Columns to match the data model
# empLength is in months - convert to years
cols [['fundedAmount','intRate','installment','empLength','annualInc','dti','delinq2Yrs','inqLast6Mths',
       'mthsSinceLastDelinq','mthsSinceLastRecord','openAcc','pubRec','revolBal','revolUtil','totalAcc',
      'collections12MthsExMed','mthsSinceLastMajorDerog','accNowDelinq','totCollAmt','totCurBal','openAcc6m',
      'openIl6m','openIl12m','openIl24m','iLUtil','maxBalBc','allUtil','totalRevHiLim','inqLast12m','accOpenPast24Mths',
      'avgCurBal','bcUtil','chargeoffWithin12Mths','delinqAmnt','moSinOldIlAcct','moSinOldRevTlOp','moSinRcntRevTlOp',
      'moSinRcntTl','mortAcc','mthsSinceRecentBc','mthsSinceRecentBcDlq','mthsSinceRecentInq','mthsSinceRecentRevolDelinq',
      'numAcctsEver120Ppd','numActvBcTl','numActvRevTl','numBcSats','numBcTl','numIlTl','numOpRevTl','numRevAccts',
      'numRevTlBalGt0','numSats','numTl120dpd2m','numTl30dpd','numTl90gDpd24m','numTlOpPast12m','pctTlNvrDlq',
      'percentBcGt75','pubRecBankruptcies','taxLiens','totHiCredLim','totalBalExMort','totalBcLimit','totalIlHighCreditLimit',
      'month','term','term_36','term_60','grade_a','grade_b','grade_c','grade_d','grade_e','grade_f','grade_g',
      '']]




In [ ]:
- no funded_amnt_inv
- changed emp_length to month
- no 'total_rec_late_fee'
- no recoveries
- no collection_recovery_fee
- no policy code
- no bc_open_to_buy
- no repaid

# Data Model
```['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'repaid',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 -'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'pymnt_plan_n',
 'pymnt_plan_y',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY']```